# **EMOTION DETECTION**

In [16]:
# Getting all dependencies

from keras.utils import to_categorical
from keras.preprocessing.image import load_img  
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
from keras.models import model_from_json
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder


In [2]:
# Setting up the directories

TRAINING_DIR = './images/train'
TEST_DIR = './images/validation'

In [3]:
# Making dataframes for the training and test data (with images and labels)

def create_dataframe(directory):
    image_paths = []
    labels = []
    
    for label in os.listdir(directory):
        label_dir = os.path.join(directory, label)
        if os.path.isdir(label_dir):
            for image_name in os.listdir(label_dir):
                image_path = os.path.join(label_dir, image_name)
                image_paths.append(image_path)
                labels.append(label)
            print(f"Completed {label_dir}")
    return image_paths, labels

In [4]:
# Making Train and Test dataframes
train_df = pd.DataFrame()
train_df['image_path'], train_df['label'] = create_dataframe(TRAINING_DIR)

test_df = pd.DataFrame()
test_df['image_path'], test_df['label'] = create_dataframe(TEST_DIR)
print("Train and test dataframes created successfully.")

print(train_df)
print(test_df)

Completed ./images/train\angry
Completed ./images/train\disgust
Completed ./images/train\fear
Completed ./images/train\happy
Completed ./images/train\neutral
Completed ./images/train\sad
Completed ./images/train\surprise
Completed ./images/validation\angry
Completed ./images/validation\disgust
Completed ./images/validation\fear
Completed ./images/validation\happy
Completed ./images/validation\neutral
Completed ./images/validation\sad
Completed ./images/validation\surprise
Train and test dataframes created successfully.
                             image_path     label
0            ./images/train\angry\0.jpg     angry
1            ./images/train\angry\1.jpg     angry
2           ./images/train\angry\10.jpg     angry
3        ./images/train\angry\10002.jpg     angry
4        ./images/train\angry\10016.jpg     angry
...                                 ...       ...
28816  ./images/train\surprise\9969.jpg  surprise
28817  ./images/train\surprise\9985.jpg  surprise
28818  ./images/train\sur

In [5]:
# Extracting the features from the dataframes

def extract_features(images):
    features = []
    for img in tqdm(images):
        image = load_img(img, color_mode='grayscale')
        image = np.array(image)
        features.append(image)
    features = np.array(features)
    features = features.reshape(features.shape[0], 48, 48, 1)
    return features

train_features = extract_features(train_df['image_path'])
test_features = extract_features(test_df['image_path'])

100%|██████████| 7066/7066 [00:03<00:00, 2171.24it/s]


In [6]:
# Normalizing the features

x_train = train_features.astype('float32') / 255.0
x_test = test_features.astype('float32') / 255.0

In [7]:
# Encoding the labels

label_encoder = LabelEncoder()
label_encoder.fit(train_df['label'])

LabelEncoder()

In [8]:
# Converting the labels to integers

y_train = label_encoder.transform(train_df['label'])
y_test = label_encoder.transform(test_df['label'])

In [9]:
# Converting the integer labels to one-hot encoded vectors
# (7 classes for the 7 emotions)

y_train = to_categorical(y_train, num_classes=7)
y_test = to_categorical(y_test, num_classes=7)

In [10]:
# Convolutional Neural Network (CNN) model
# The model is a sequential model with convolutional layers, max pooling layers, and dropout layers
# The model is designed to classify images into 7 different classes (emotions)

model =  Sequential()
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1)))
mpder = MaxPooling2D(pool_size=(2, 2))
model.add(Dropout(0.4))

model.add(Conv2D(256, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))

model.add(Flatten())

# Fully connected layers

model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.4))

# Output layer
model.add(Dense(7, activation='softmax'))

In [11]:
# Configuring the model

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, batch_size=128, epochs=100, validation_data=(x_test, y_test))


Epoch 1/100
226/226 [==============================] - 543s 2s/step - loss: 1.8222 - accuracy: 0.2495 - val_loss: 1.7132 - val_accuracy: 0.3121
Epoch 2/100
226/226 [==============================] - 603s 3s/step - loss: 1.5970 - accuracy: 0.3697 - val_loss: 1.4031 - val_accuracy: 0.4595
Epoch 3/100
226/226 [==============================] - 591s 3s/step - loss: 1.4138 - accuracy: 0.4560 - val_loss: 1.3267 - val_accuracy: 0.4994
Epoch 4/100
179/226 [======================>.......] - ETA: 2:04 - loss: 1.3227 - accuracy: 0.4928

KeyboardInterrupt: 

In [12]:
# Saving the model

model_json = model.to_json()
with open("emotionDetector.json", "w") as json_file:
    json_file.write(model_json)
model.save("emotionDetector.h5")
print("Model saved successfully.")

c:\Users\abdul\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Model saved successfully.


In [17]:
# Load Model

json_file = open('emotionDetector.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights("emotionDetector.h5")

In [30]:
# Checking the model output

label = ['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']

def predict_emotion(image):
    image = load_img(image, color_mode='grayscale')
    image = np.array(image)
    image = image.reshape(1, 48, 48, 1)
    image = image / 255.0
    
    return image

# Test 1
image = './images/validation/angry/38.jpg'
print("Original image is of angry")
image = predict_emotion(image)
predicted = loaded_model.predict(image)
print("Model Prediction is: ",label[np.argmax(predicted)])

# Test 2
image = './images/validation/sad/20.jpg'
print("Original image is of sad")
image = predict_emotion(image)
predicted = loaded_model.predict(image)
print("Model Prediction is: ",label[np.argmax(predicted)])

Original image is of angry
1/1 [==============================] - 0s 30ms/step
Model Prediction is:  angry
Original image is of sad
1/1 [==============================] - 0s 25ms/step
Model Prediction is:  sad
